In [48]:

def process_line(line):
    if line[0]=="#":
        return None
    line = line.strip()
    if "#" in line:
        line = line[:line.index("#")]
    line = line.strip()
    line = line.split(" ")
    line = line[0].split("\t") + line[1:]
    if len(line)==1:
        return None
    return line

from collections import defaultdict
rules = list()
filename = 'grammar.gr.txt'
dict_prob = defaultdict(list)
dict_rhs = defaultdict(list)
# RHS: right hand side
with open(filename, 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = process_line(line)
        if not line:
            continue
        weight = line[0]
        lhs = line[1]
        rhs = line[2:]
        dict_prob[lhs].append(weight)
        dict_rhs[lhs].append(rhs)
        rules.append([weight, lhs, rhs])

In [49]:
dict_prob

defaultdict(list,
            {'ROOT': ['1', '1', '1'],
             'S': ['1'],
             'VP': ['1'],
             'NP': ['1', '1'],
             'PP': ['1'],
             'Noun': ['1', '1', '1', '1', '1', '1'],
             'Verb': ['1', '1', '1', '1', '1'],
             'Det': ['1', '1', '1'],
             'Adj': ['1', '1', '1', '1'],
             'Prep': ['1', '1', '1', '1']})

In [50]:
dict_rhs

defaultdict(list,
            {'ROOT': [['S', '.'],
              ['S', '!'],
              ['is', 'it', 'true', 'that', 'S', '?']],
             'S': [['NP', 'VP']],
             'VP': [['Verb', 'NP']],
             'NP': [['Det', 'Noun'], ['NP', 'PP']],
             'PP': [['Prep', 'NP']],
             'Noun': [['Adj', 'Noun'],
              ['president'],
              ['sandwich'],
              ['pickle'],
              ['chief', 'of', 'staff'],
              ['floor']],
             'Verb': [['ate'],
              ['wanted'],
              ['kissed'],
              ['understood'],
              ['pickled']],
             'Det': [['the'], ['a'], ['every']],
             'Adj': [['fine'], ['delicious'], ['perplexed'], ['pickled']],
             'Prep': [['with'], ['on'], ['under'], ['in']]})

In [51]:
rules

[['1', 'ROOT', ['S', '.']],
 ['1', 'ROOT', ['S', '!']],
 ['1', 'ROOT', ['is', 'it', 'true', 'that', 'S', '?']],
 ['1', 'S', ['NP', 'VP']],
 ['1', 'VP', ['Verb', 'NP']],
 ['1', 'NP', ['Det', 'Noun']],
 ['1', 'NP', ['NP', 'PP']],
 ['1', 'PP', ['Prep', 'NP']],
 ['1', 'Noun', ['Adj', 'Noun']],
 ['1', 'Verb', ['ate']],
 ['1', 'Verb', ['wanted']],
 ['1', 'Verb', ['kissed']],
 ['1', 'Verb', ['understood']],
 ['1', 'Verb', ['pickled']],
 ['1', 'Det', ['the']],
 ['1', 'Det', ['a']],
 ['1', 'Det', ['every']],
 ['1', 'Noun', ['president']],
 ['1', 'Noun', ['sandwich']],
 ['1', 'Noun', ['pickle']],
 ['1', 'Noun', ['chief', 'of', 'staff']],
 ['1', 'Noun', ['floor']],
 ['1', 'Adj', ['fine']],
 ['1', 'Adj', ['delicious']],
 ['1', 'Adj', ['perplexed']],
 ['1', 'Adj', ['pickled']],
 ['1', 'Prep', ['with']],
 ['1', 'Prep', ['on']],
 ['1', 'Prep', ['under']],
 ['1', 'Prep', ['in']]]

In [52]:
import random

class PCFG:
    def __init__(self, dict_prob, dict_rhs, max_expansions=450):
        self.dict_prob = dict_prob  
        self.dict_rhs = dict_rhs
        self.max_expansions = max_expansions
        self.expansion_count = 0
    
    def sample(self, symbol):
        # Base case: If the symbol is a terminal, return it
        if symbol not in self.dict_rhs:
            return symbol
        
        # It should just print "..." if M is reached
        if self.expansion_count >= self.max_expansions:
            return "..."
        
        # Increment expansion count
        self.expansion_count += 1
        
        # Select a rule based on weights
        rule = self._weighted_choice(self.dict_rhs[symbol], self.dict_prob[symbol])
        
        # Recursively expand each symbol in the selected rule
        result = []
        for sub_symbol in rule:
            result.append(self.sample(sub_symbol))
        
        return " ".join(result)
    
    def _weighted_choice(self, rules, weights):
        total = sum(float(w) for w in weights)
        # Get distribution
        normalized_weights = [float(w) / total for w in weights]

        rand_val = random.uniform(0, 1)
        cumulative_weight = 0
        for rule, weight in zip(rules, normalized_weights):
            cumulative_weight += weight
            if rand_val <= cumulative_weight:
                return rule
    
    def generate_sentence(self):
        self.expansion_count = 0
        return self.sample("ROOT")

# M = 450
pcfg = PCFG(dict_prob, dict_rhs, max_expansions=450)

# Generate a sentence
sentence = pcfg.generate_sentence()
print("Generated sentence:", sentence)

Generated sentence: a pickle on a chief of staff under a president with the pickle under a delicious chief of staff in every fine pickled pickle on the pickle on a chief of staff on the chief of staff in the chief of staff in the floor in every sandwich in a chief of staff with a pickle on the president in the president on the pickle in the chief of staff in a president on the pickle under the pickle under every sandwich with the chief of staff under every fine chief of staff on the chief of staff with every chief of staff on every president under the president under every chief of staff with every president under every pickle on a president on every chief of staff with a president in the president with the sandwich in the president in the delicious pickle on every pickle on the sandwich on a sandwich under a chief of staff in the floor with the fine chief of staff on a president under a chief of staff on a sandwich in a president in every president with the chief of staff on the presi